<a href="https://colab.research.google.com/github/DhivyaMadhavan/Emotion_Recognition/blob/main/Emotion_Recognition_dlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.7 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=5afc8aaa3c5eee253712bb80737334ebf4e175af5fdb50636fcccd37b73aec2b
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [4]:
import cv2
import numpy as np
import dlib
from deepface import DeepFace

def analyze_video_emotions_dlib(video_path):
    # Initialize video capture
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video stream.")
        return

    # Get total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in the video: {total_frames}")

    # Initialize emotion detection model
    emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    emotion_count = {label: 0 for label in emotion_labels}

    # Load Dlib's CNN face detector model
    detector = dlib.cnn_face_detection_model_v1('/content/mmod_human_face_detector.dat')

    # Function to update emotion count
    def update_emotion_count(pred):
        for emotion, score in pred.items():
            emotion_count[emotion] += score

    frame_index = 0
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or failed to capture frame")
            break

        if frame_index % 100 == 0:
            # Convert frame to RGB (Dlib expects RGB images)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detect faces using Dlib
            detections = detector(rgb_frame)

            if len(detections) == 0:
                print(f"No face detected in frame {frame_index}")
            else:
                # Process each detected face
                for detection in detections:
                    x, y, w, h = (detection.rect.left(), detection.rect.top(), detection.rect.right(), detection.rect.bottom())
                    face_points = rgb_frame[y:h, x:w]

                    # Check if the face is large enough
                    if face_points.shape[0] > 0 and face_points.shape[1] > 0:
                        # Resize face image to 48x48
                        resized_face = cv2.resize(face_points, (48, 48), interpolation=cv2.INTER_AREA)

                        # Convert face image to grayscale
                        resized_face_gray = cv2.cvtColor(resized_face, cv2.COLOR_RGB2GRAY)

                        # Normalize and reshape
                        normalized_face = resized_face_gray / 255.0
                        reshaped_face = normalized_face.reshape(1, 48, 48, 1)

                        # Analyze emotions
                        try:
                            predictions = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
                            for pred in predictions:
                                update_emotion_count(pred['emotion'])
                            frame_count += 1
                        except Exception as e:
                            print(f"Error analyzing frame {frame_index}: {e}")

        frame_index += 1

    # Release the capture and close all OpenCV windows
    cap.release()

    # Compute average summary of emotions
    total_emotions = sum(emotion_count.values())
    average_emotions = {label: (count / total_emotions) if total_emotions > 0 else 0 for label, count in emotion_count.items()}

    # Print the average summary of emotions
    print("Average summary of emotions over every 100th frame:")
    for emotion, avg in average_emotions.items():
        print(f"{emotion}: {avg:.2f}")

    # Find and print the predominant emotion
    if total_emotions > 0:
        predominant_emotion = max(emotion_count, key=emotion_count.get)
        print(f"\nPredominant Emotion: {predominant_emotion}")
    else:
        print("\nNo emotions detected.")

    # Display the frame count
    print(f"Total frames processed: {frame_count}")

# Example usage:
video_path = "/content/demo.mp4"
analyze_video_emotions_dlib(video_path)


Total frames in the video: 13497
24-08-08 05:28:20 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 291MB/s]


End of video or failed to capture frame
Average summary of emotions over every 100th frame:
angry: 0.21
disgust: 0.01
fear: 0.08
happy: 0.02
sad: 0.38
surprise: 0.00
neutral: 0.30

Predominant Emotion: sad
Total frames processed: 135
